In [2]:
#installing Biopython to assist with accessing API to obtain data from PubMed
!pip install biopython

In [3]:
#setting up filenames
absts_filename = "cardiac_absts.txt"
absts_annotations_filename = "cardiac_absts_pubtator_annotations.xml"
reports_filename = "cardiac_fulltexts.xml"
reports_annotations_filename = "cardiac_fulltexts_pubtator_annotations.xml"


In [4]:
#downloading set of PubMed records through the API
from Bio import Entrez
Entrez.email = "kbali01@g.ucla.edu"  # Write your email address in between the quotes
search_term = "((cardiac) AND (proteome)AND Adults[MeSH Terms])"
handle = Entrez.esearch(db="pubmed", term=search_term, retmax=200) #We don't know how many results there will be, but we can set retmax to up to 10,000 to get that many results.
record = Entrez.read(handle)
handle.close()

#prints the list of PubMed IDs
print(record)
idlist = record['IdList']
print(idlist)

{'Count': '233', 'RetMax': '200', 'RetStart': '0', 'IdList': ['32938215', '32806897', '32704130', '32487995', '32427855', '32385381', '32178279', '32117548', '31988066', '31918764', '31862877', '31844116', '31825151', '31819116', '31806903', '31783201', '31734660', '31585756', '31562990', '31494943', '31476444', '31434864', '31286493', '31242583', '31204432', '31186442', '31104495', '31092728', '31056672', '30928653', '30922315', '30907085', '30887811', '30874584', '30870787', '30838756', '30836973', '30806752', '30805593', '30782942', '30742448', '30739867', '30658193', '30571344', '30562113', '30496173', '30478197', '30451909', '30368668', '30347343', '30332462', '30261322', '30255987', '30248148', '30224653', '30212933', '30201848', '30192561', '30002058', '29956481', '29915101', '29854824', '29853569', '29784904', '29717219', '29682908', '29676857', '29665821', '29649363', '29540862', '29505607', '29484645', '29369166', '29320704', '29258991', '29258006', '29251807', '29235871', '2

In [5]:
#using the API's EFetch function to retrieve records for each of the PubMed IDs
#entries are obtained in the MEDLINE format
from Bio import Medline
handle = Entrez.efetch(db="pubmed", id=idlist, rettype="medline", retmode = "text")
pm_recs = Medline.parse(handle)
pm_recs_list = list(pm_recs)
for rec in pm_recs_list:
  print(rec)

{'PMID': '32938215', 'OWN': 'NLM', 'STAT': 'MEDLINE', 'DCOM': '20201214', 'LR': '20201214', 'IS': '1524-4636 (Electronic) 1079-5642 (Linking)', 'VI': '40', 'IP': '11', 'DP': '2020 Nov', 'TI': 'Free Thiol beta2-GPI (beta-2-Glycoprotein-I) Provides a Link Between Inflammation and Oxidative Stress in Atherosclerotic Coronary Artery Disease.', 'PG': '2794-2804', 'LID': '10.1161/ATVBAHA.120.315156 [doi]', 'AB': 'OBJECTIVE: Atherosclerotic coronary artery disease is well recognised as an inflammatory disorder that is also influenced by oxidative stress. beta2-GPI (beta-2-glycoprotein-I) is a circulating plasma protein that undergoes post-translational modification and exists in free thiol as well as oxidized forms. The aim of this study was to assess the association between these 2 post-translational redox forms of beta2-GPI and atherosclerotic coronary artery disease. Approach and Results: Stable patients presenting for elective coronary angiography or CT coronary angiography were prospecti

In [6]:
#saving the title and abstract for each to a file
with open(absts_filename, "w") as outfile:
  for rec in pm_recs_list:
    title = rec.get('TI')
    abst = rec.get('AB', "No Abstract")
    outfile.write(title + "\n" + abst + "\n\n")

In [7]:
#retrieving full-text documents from PubMed Central
pmc_ids = []
for rec in pm_recs_list:
  pmc_ids.append(rec.get('PMC'))
  pmc_ids_filtered = list(filter(None, pmc_ids)) 
print("%s out of %s entries have PMC IDs." % (len(pmc_ids_filtered), len(pm_recs_list)))
print(pmc_ids_filtered)

107 out of 200 entries have PMC IDs.
['PMC7489050', 'PMC7378184', 'PMC7266817', 'PMC7237418', 'PMC7211010', 'PMC7146314', 'PMC7031431', 'PMC6953276', 'PMC6925199', 'PMC6914779', 'PMC6901481', 'PMC7062043', 'PMC6627068', 'PMC6560130', 'PMC6542631', 'PMC6437869', 'PMC6558644', 'PMC6420620', 'PMC6430080', 'PMC6484326', 'PMC6399814', 'PMC6398744', 'PMC6369230', 'PMC6403451', 'PMC6264811', 'PMC6475111', 'PMC6242904', 'PMC6263790', 'PMC6192629', 'PMC6265639', 'PMC6152976', 'PMC6284793', 'PMC6156053', 'PMC6196832', 'PMC6011234', 'PMC5949160', 'PMC6023756', 'PMC5962559', 'PMC5931620', 'PMC6104105', 'PMC5905170', 'PMC6162093', 'PMC5837105', 'PMC5978942', 'PMC5869115', 'PMC5849518', 'PMC5813793', 'PMC5719427', 'PMC5695135', 'PMC5880582', 'PMC5553757', 'PMC5550488', 'PMC5586413', 'PMC5515959', 'PMC5573768', 'PMC5564650', 'PMC5342174', 'PMC5335613', 'PMC5314136', 'PMC5834230', 'PMC5053516', 'PMC7101709', 'PMC4911082', 'PMC4882859', 'PMC4833206', 'PMC4719542', 'PMC4707500', 'PMC4664895', 'PMC464449

In [8]:
#removing "PMC" from the PMC IDs before fetching records
pmc_ids_trim = [id[3:] for id in pmc_ids_filtered]

In [9]:
import xml.dom.minidom

In [10]:
handle = Entrez.efetch(db="pmc", id=pmc_ids_trim, rettype='full', retmode='xml')
hxml = xml.dom.minidom.parseString(handle.read())
hxml_pretty = hxml.toprettyxml()

with open(reports_filename, "w") as outfile:
  for line in hxml_pretty:
    outfile.write(line)

In [11]:
#using Python's request package to obtain annotations of named entities from PubTator Central in BioC XML format
import requests, xml.dom.minidom

In [12]:
#obtaining annotations for the PubMed Central full texts

#pmc_ids_joined = ",".join(pmc_ids_filtered)
pmc_ids_joined = ",".join(pmc_ids_filtered[:50])
r = requests.get("https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocxml?&concepts=gene&pmcids=" + pmc_ids_joined)
rxml_reports = xml.dom.minidom.parseString(r.text)
rxml_pretty_reports = rxml_reports.toprettyxml()

with open(reports_annotations_filename, "w") as outfile:
  for line in rxml_pretty_reports:
    outfile.write(line)

In [13]:
#And some quick file cleaning.
with open(reports_annotations_filename, "r") as readfile:
  lines = readfile.readlines()
with open(reports_annotations_filename, "w") as outfile:
  for line in lines:
    if line.strip("\n") not in ["<?xml version=\"1.0\" ?>", 
                                "<!DOCTYPE collection",
                                "  SYSTEM 'BioC.dtd'>"]:
      outfile.write(line)

In [14]:
#retrieving annotations for all PubMed entries now

#First, a small function to help with preparing batches.
def batch_this(mylist, n=1):
  length = len(mylist)
  for ndx in range(0, length, n):
    yield mylist[ndx:min(ndx + n, length)]

In [15]:
for batch in batch_this(idlist,100): 
  pmids_joined = ",".join(batch)
  r = requests.get("https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocxml?&concepts=gene&pmids=" + pmids_joined)
  rxml_absts = xml.dom.minidom.parseString(r.text)
  rxml_pretty_absts = rxml_absts.toprettyxml()

  with open(absts_annotations_filename, "a") as outfile:
    for line in rxml_pretty_absts:
      outfile.write(line)

In [16]:
#Need to clean up the XML output a bit to join.
with open(absts_annotations_filename, "r") as readfile:
  lines = readfile.readlines()
with open(absts_annotations_filename, "w") as outfile:
  seenheaders = []
  for line in lines:
    line_ok = True
    if line.strip("\n") in ["<collection>",
                            "	<source>PubTator</source>",
                            "	<date/>",
                            "	<key>BioC.key</key>"]:
      if line in seenheaders:
        line_ok = False
      else:
        seenheaders.append(line)
    if line.strip("\n") == "</collection>":
        line_ok = False
    if line.strip("\n") in ["<?xml version=\"1.0\" ?>", 
                            "<!DOCTYPE collection",
                            "  SYSTEM 'BioC.dtd'>"]:
        line_ok = False
    if line_ok:
      outfile.write(line)
  outfile.write("</collection>\n")

In [17]:
#currently have two sets of annotations: one for abstracts and one for full texts in BioC XML format

import xml.etree.ElementTree as ET
from collections import Counter, OrderedDict
corpus_filenames = [reports_annotations_filename, absts_annotations_filename]

In [24]:
#now counting total number of unique annotations for each set, per set and per document

fulltext_gene_ids = [] #will hold all the gene ids found across all fulltexts
abstract_gene_ids = [] #will hold all the gene ids found across all abstracts

for corpus in corpus_filenames:
  print("\n* Reading from " + corpus)
  tree = ET.parse(corpus)
  root = tree.getroot()
  
  print("\n** Gene frequency across the entire corpus:")
  corpus_annotations = []
 
  for text in root.findall("./document/passage/annotation/infon[1]"):
    #We ignore capitalization to merge counts where it varies
    corpus_annotations.append((text.text).lower())
  #Have to get counts of each term's occurrence
  corpus_annotations_counts = dict(Counter(corpus_annotations))
  sorted_corpus_annotations_counts = sorted(corpus_annotations_counts.items(), 
                                            key=lambda kv: kv[1],
                                            reverse = True)
  for count in sorted_corpus_annotations_counts:
    if corpus == "cardiac_fulltexts_pubtator_annotations.xml":    
        fulltext_gene_ids.append(count[0])
    if corpus == "cardiac_absts_pubtator_annotations.xml":
        abstract_gene_ids.append(count[0])
    print(count[0] +": " + str(count[1]))
    
    
  if corpus == "cardiac_fulltexts_pubtator_annotations.xml":
    print("\n** Gene frequency in each document (identified by PubMed Central ID):")
    
  if corpus == "cardiac_absts_pubtator_annotations.xml":
    print("\n** Gene frequency in each document (identified by PubMed ID):")
    
  for document in root.findall("document"):
    pmid = document.find('id').text
    document_annotations = []
    for annotation in document.iter('annotation'):
        document_annotations.append(annotation[0].text)
    #Now to get counts
    doc_annotations_counts = dict(Counter(document_annotations))
    sorted_doc_annotations_counts = sorted(doc_annotations_counts.items(), 
                                            key=lambda kv: kv[1],
                                            reverse = True)
    print(pmid + ": " + str(sorted_doc_annotations_counts))
    
    


* Reading from cardiac_fulltexts_pubtator_annotations.xml

** Gene frequency across the entire corpus:
5034: 327
335: 157
18129: 134
4353: 125
3620: 116
1756: 111
1401: 105
722: 98
9242: 96
5284: 84
7273: 81
6908: 79
2244: 74
51206: 74
30968: 70
348: 68
3630: 65
38: 65
3075: 63
7040: 62
338: 61
31293: 58
4879: 56
151648: 53
345: 51
11102: 47
56267: 46
2934: 44
3700: 43
152330: 41
133522: 40
183: 39
6288: 37
17880: 37
59284: 37
3934: 36
3827: 35
4312: 35
718: 35
4311: 34
699: 33
213: 33
116519: 32
5496: 31
56925: 31
1: 31
50848: 31
55937: 31
3250: 31
337: 30
3303: 30
1003: 30
7547: 29
2828: 29
170589: 29
4316: 29
820: 28
7058: 28
3053: 28
6714: 27
351: 27
2670: 27
3816: 26
966: 26
7124: 26
7412: 26
6876: 26
1012: 26
100130342: 25
7276: 25
1628: 24
133: 23
1509: 22
5345: 22
207: 21
9575: 21
3952: 21
2: 21
3918: 20
3482: 20
3494: 19
341: 19
7057: 19
1803: 19
8550: 19
4846: 18
336: 18
7448: 18
2335: 18
6696: 18
3933: 18
3080: 18
728: 18
5265: 17
18131: 17
5624: 17
1351: 16
5465: 16
5444: 

6019: 1
284;285: 1
8038: 1
2735: 1
2833: 1
6373: 1
1236: 1
6366: 1
7042: 1
55532: 1
1306: 1
51535: 1
64167: 1
23768: 1
92: 1
5457: 1
9033: 1
6558: 1
775: 1
785: 1
766: 1
64116: 1
2250: 1
2254: 1
81575: 1
10266: 1
11318: 1
115557: 1
4140: 1
56034: 1
7201: 1
134: 1
2555: 1
5139: 1
6523: 1
1453: 1
2077: 1
11535: 1
12310: 1
54598: 1
72: 1
1890: 1
322275: 1
4682: 1
23352: 1
407: 1
5177: 1
3552;3562;3567;3576: 1
7040;21803: 1
81693: 1
326: 1
4585: 1
545370: 1
21803: 1
6406: 1
367: 1
668478: 1
16017: 1
10678: 1
4069: 1
51266: 1
100302740: 1
5308: 1
21825: 1
6310: 1
9429: 1
16590: 1
5015: 1
6615: 1
3651: 1
64321: 1
176: 1
4074: 1
4924: 1
5351: 1
57554: 1
23037: 1
25977: 1
6709: 1
3675: 1
58513: 1
23516: 1
133957: 1
20: 1
2043: 1
57582: 1
64398: 1
7436: 1
80727: 1
6541: 1
8793: 1
57003: 1
84059: 1
3673: 1
7433: 1
8325: 1
59284;50848: 1
1002: 1
3384: 1
8482: 1
12562: 1
16456: 1
6531: 1
26269: 1
2990: 1
3028: 1
5420: 1
1297: 1
3480: 1
131450: 1
648: 1
10411: 1
54386: 1
7296: 1
7057;2153: 1
5269: 


** Gene frequency across the entire corpus:
335: 279
7276: 225
3075: 216
348: 189
4353: 162
5034: 135
6287: 126
5265: 124
1674: 117
2244: 117
3240: 117
8216: 108
1191: 108
4780: 90
6279: 81
18129: 72
5444: 72
5216: 72
2023: 63
345: 63
3620: 63
3308: 63
947: 63
494326: 63
1401: 55
10631: 54
5950: 54
6439: 54
133522: 54
152330: 54
7547: 54
7273: 54
5500: 54
51206: 54
3315: 54
5473: 54
723814: 54
7052: 54
350: 50
3934: 45
7422: 45
2335: 45
5315: 45
336: 45
857: 45
6280: 45
11816: 45
100329167: 45
1728: 45
6120: 45
5284: 45
26762: 36
811: 36
338: 36
12: 36
4879: 36
2: 36
4311: 36
5894: 36
3630: 36
6908: 36
151648: 36
50848: 36
7431: 36
4321: 36
721: 36
3329: 36
53369: 36
10935: 36
462: 29
9370: 28
3320: 27
344: 27
3569: 27
3371: 27
11346: 27
3303: 27
4633: 27
7018: 27
1756: 27
7412: 27
183: 27
7124: 27
4345: 27
2828: 27
3080: 27
5321: 27
4069: 27
718: 27
3250: 27
25932: 27
773: 27
6238: 27
6289: 27
1471: 20
2638: 20
5972: 20
2273: 18
1805: 18
729359: 18
3827: 18
4629: 18
3309: 18
4723: 18

27624754: [('2023', 6), ('5315', 5), ('811', 1), ('4629', 1), ('100507027', 1)]
27711126: [('3320', 2), ('2869', 1), ('336', 1), ('3502', 1), ('309', 1), ('509', 1), ('2949', 1), ('10063', 1), ('857', 1), ('306', 1), ('2805', 1), ('8659', 1), ('5162', 1), ('622', 1), ('66035', 1), ('3326', 1), ('94239', 1), ('3309', 1), ('4723', 1)]
27908912: []
28196416: [('7276', 4), ('5950', 3)]
28209220: [('348', 9), ('345', 6), ('344', 3), ('335', 2), ('338', 2), ('336', 1), ('55937', 1)]
28223106: [('338', 1)]
28256566: [('5465', 1), ('5595;5594', 1)]
28273075: []
28329269: [('12', 2), ('10631', 1), ('1462', 1)]
28341603: [('1674', 11)]
28427148: [('6271', 1), ('1938', 1)]
28437237: [('627', 1), ('92737', 1), ('6364', 1), ('4314', 1), ('690', 1), ('26291', 1), ('3593', 1), ('5617', 1), ('7984', 1), ('3569', 1), ('5806', 1), ('6401', 1)]
28501389: [('335', 1), ('128876', 1)]
28624389: []
28679606: []
28720870: [('3685', 2), ('2335', 2), ('3371', 2), ('7448', 1)]
28784649: []
28794502: []
28800743:

26653129: []
26688386: [('642658', 1), ('3673', 1), ('320', 1)]
26750556: []
26792056: [('6279', 6)]
26834087: [('6120', 5)]
26898369: [('6696', 1)]
27084846: [('7052', 6), ('1509', 2), ('1808', 2), ('1200', 2), ('5265', 1)]
27095521: [('8935', 2), ('7057', 1)]
27122311: [('50802', 2), ('3263', 1), ('7018', 1), ('3240', 1)]
27220540: []
27230659: [('3934', 4), ('26762', 4)]
27308822: [('5265', 1)]
27363418: [('7422', 3)]
27412778: [('59272', 1), ('10631', 1), ('1490', 1), ('2335', 1)]
27429122: []
27450324: [('55752', 1), ('2273', 1), ('1805', 1)]
27624754: [('2023', 6), ('5315', 5), ('811', 1), ('4629', 1), ('100507027', 1)]
27711126: [('3320', 2), ('2869', 1), ('336', 1), ('3502', 1), ('309', 1), ('509', 1), ('2949', 1), ('10063', 1), ('857', 1), ('306', 1), ('2805', 1), ('8659', 1), ('5162', 1), ('622', 1), ('66035', 1), ('3326', 1), ('94239', 1), ('3309', 1), ('4723', 1)]
27616206: [('6279', 3), ('729359', 2), ('3827', 2)]
27908912: []
28196416: [('7276', 4), ('5950', 3)]
28209220:

In [19]:
#converted the uniprot IDs of the heart tissue proteins from The Human Protein Atlas into gene IDs and stored in a csv

#this code is reading this csv and putting the gene IDs into an array for the next comparison
import csv
with open('heart_tissue_gene_ids - Sheet1.csv') as csvfile:
    readCSV = csv.reader(csvfile)
    
    heart_tissue_gene_ids = []
    
    for row in readCSV:
        heart_tissue_gene_ids.append(row[0])

In [20]:
#way the intersection between two lists will be found
def intersection(lst1, lst2):
    temp = set(lst2)
    lst3 = [value for value in lst1 if value in temp]
    return lst3

#these will hold the gene ids that are in the intersections
fulltext_intersection = []
abstract_intersection = []

#finding the intersection between the two lists
fulltext_intersection = intersection(fulltext_gene_ids, heart_tissue_gene_ids)
abstract_intersection = intersection(abstract_gene_ids, heart_tissue_gene_ids)

#printing intersection of fulltext list and heart tissue list
print("\n **Genes that are in both fulltexts list and heart tissue list from The Human Protein Atlas:")

print(fulltext_intersection)
print("*The number of gene ids in this list is:")
print(len(fulltext_intersection))

#printing intersection of abstract list and heart tissue list
print("\n **Genes that are in both abstract list and heart tissue list from The Human Protein Atlas:")

print(abstract_intersection)
print("*The number of gene ids in this list is:" )
print(len(abstract_intersection))


 **Genes that are in both fulltexts list and heart tissue list from The Human Protein Atlas:
['7273', '4879', '9377', '2170', '6262', '5196', '7139', '488', '54205', '282996', '3764', '4878', '3270', '6546', '2626', '26577', '1428', '4634', '7137', '5350', '1410', '1000', '10627', '4512', '4190', '816', '127294', '51422', '84675', '55600', '775', '5139', '4023', '57644', '9086']
*The number of gene ids in this list is:
35

 **Genes that are in both abstract list and heart tissue list from The Human Protein Atlas:
['1674', '7273', '4879', '11346', '4633', '4634', '22808', '2805', '6271', '4635', '58498', '10611', '153', '7168', '5730', '488', '8988']
*The number of gene ids in this list is:
17
